# Propotional Integral Derivative Control - Mini Lesson
The lesson is took from : https://ctms.engin.umich.edu/CTMS/index.php?example=Introduction&section=ControlPID

In [ ]:
import control as ct
import numpy as np
import matplotlib.pyplot as plt

## Create a Linear LTI (Linear Time Invariant) SISO (Single Input Single Output) System

General Control Scheme

![image.png](imgs/feedback_block.png)

PID Controller will replace C(s) block here.

PID controller equation as below:

$$u(t) = K_p * e(t) + K_i * \int_{a}^{b} e(t)dt + K_d * \frac{d e(t)}{dt}

Let's convert it to Laplace domain

![image.png](imgs/Introduction_ControlPID_eq.png)

Write the equation in control library

In [ ]:
from env_control import create_pid
create_pid(1,1,1)


!***Warning***: It is impossible to realize in time-domain since the term derivative breaks causality. 
In this case an un-effective zero is added to system to very very far to origin

## Meaning of P,I,D Terms

- $K_p$: Increasing the proportional gain ($K_p$) has the effect of proportionally increasing the control signal for the same level of error. The fact that the controller will "push" harder for a given level of error tends to cause the closed-loop system to react more quickly, but also to overshoot more. Another effect of increasing $K_p$ is that it tends to reduce, but not eliminate, the steady-state error.
- $K_d$ : adds the ability of the controller to "anticipate" error. With simple proportional control, if $K_p$ is fixed, the only way that the control will increase is if the error increases. With derivative control, the control signal can become large if the error begins sloping upward, even while the magnitude of the error is still relatively small. This anticipation tends to add damping to the system, thereby decreasing overshoot. The addition of a derivative term, however, has no effect on the steady-state error.
- $K_i$ : tends to help reduce steady-state error. If there is a persistent, steady error, the integrator builds and builds, thereby increasing the control signal and driving the error down. A drawback of the integral term, however, is that it can make the system more sluggish (and oscillatory) since when the error signal changes sign, it may take a while for the integrator to "unwind."

| CL RESPONSE | RISE TIME | OVERSHOOT | SETTLING TIME | S-S ERROR |
| ---         | ---       | ---       | ---           | ---       |
|*$$K_p$$*           | Decrease  | Increase  | Small Change  | Decrease  |
|*$$K_i$$*           | Decrease  | Increase  | Increase  | Decrease  |
|*$$K_d$$*         | Small Change  | Decrease  | Decrease  | No Change  |


## Example Problem

Suppose we have a simple mass-spring-damper system.

![image.png](imgs/mass_spring_damper.png)

The governing equation of this system is:

![image.png](imgs/mass_spring_damper_eq.png)

Taking the Laplace transform of the governing equation, we get

![image.png](imgs/mass_spring_damper_eq_laplace.png)

The transfer function between the input force $F(s)$ and the output displacement $X(s)$ then becomes:

![image.png](imgs/msd_tf.png)



Let system have parameters
- m = 1 kg
- b = 10 N s/m
- k = 20 N/m
- F = 1 N

Design a PID controller such that:
- Fast rise time
- Min overshoot
- Zero steady-state error

In [ ]:
def create_msd_system(m,b,k):
    """Create Mass-Spring-Damper System 

    Parameters
    ----------
    m : float
        mass
    b : float
        damping ratio
    k : float
        spring coefficient
        
    Returns
    -------
    _type_
        Transfer Function of Mass-Spring-Damper System in s-domain
    """
    return 1/(m*s**2 + b*s + k)

In [ ]:
sys_msd = create_msd_system(1,10,20)
sys_msd

### Open-Loop Step Response

In [ ]:
from env_control import ConfigSISO, LinearSISOEnv

# Masss-Spring-Damper system
config_siso = ConfigSISO(
        action_space=[-1,1],
        obs_space=[-10,10],
        num=[1],
        den=[1,10,20],
        x_0=[0],
        dt=0.1,
        y_0=0,
        t_0=0,
        t_end=10,
        y_ref=5)
env = LinearSISOEnv(config_siso)
env.reset()
env.open_loop_step_response()
env.render()

## PID Control Results

### P Only Control: Would lead to steady-state error

In [ ]:
config_siso = ConfigSISO(
        action_space=[-1,1],
        obs_space=[-10,10],
        num=[1],
        den=[1,10,20],
        x_0=[0],
        dt=0.05,
        y_0=0,
        t_0=0,
        t_end=3,
        y_ref=1)

env = LinearSISOEnv(config_siso)

env.reset()

c_p = create_pid(300,0,0)
print(c_p)
# G'(s) = G(s)*C(s)
env.sys = ct.series(env.sys, c_p) 
env.closed_loop_step_response()
env.render()
print(f"Final output: {env.y} // Steady-state error: {1-env.y}")

## PI Controller

In [ ]:
config_siso = ConfigSISO(
        action_space=[-1,1],
        obs_space=[-10,10],
        num=[1],
        den=[1,10,20],
        x_0=[0],
        dt=0.01,
        y_0=0,
        t_0=0,
        t_end=1,
        y_ref=3)
env = LinearSISOEnv(config_siso)
c_p = create_pid(300,15,0)
env.sys = ct.series(env.sys, c_p) 
env.closed_loop_step_response()
env.render()
print(f"Final output: {env.y} // Steady-state error: {1-env.y}")

Still has problems and rise time is low

### Add D-Term: PID

In [ ]:
config_siso = ConfigSISO(
        action_space=[-1,1],
        obs_space=[-10,10],
        num=[1],
        den=[1,10,20],
        x_0=[0],
        dt=0.01,
        y_0=0,
        t_0=0,
        t_end=1,
        y_ref=3)
env = LinearSISOEnv(config_siso)
c_p = create_pid(300,15,200)
env.sys = ct.series(env.sys, c_p) 
env.closed_loop_step_response()
env.render()
print(f"Final output: {env.y} // Steady-state error: {1-env.y}")

### Resources

- https://ctms.engin.umich.edu/CTMS/index.php?example=Introduction&section=ControlPID